In [1]:
import os

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
os.environ["LANGCHAIN_TRACING_V2"] = LANGCHAIN_TRACING_V2

LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT

GROQ_API_KEY  = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

Load the embedding model

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="model/embedding-001")


Load the llms

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm_google = ChatGoogleGenerativeAI(model = "gemini-1.5-pro" , temperature = 0 , convert_system_message_to_human=True)

from langchain_groq import ChatGroq
llm_groq = ChatGroq(model="Gemma2-9b-It")

SIMPLE AI ASSISTANT / CHATBOT

In [12]:
while True:
    question = input("Hey what u want to ask else quit")
    if question == "quit":
        print("Bye")
        break
    else:
        response = llm_google.invoke(question)
        print(response.content)

/Users/akashjain/Desktop/LangGraph/venvLanggraph/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Hi there! How can I help you today?


/Users/akashjain/Desktop/LangGraph/venvLanggraph/lib/python3.9/site-packages/langchain_google_genai/chat_models.py:367: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I am doing well, thank you for asking. How are you?
Bye


Now lets also add memory thing in this


In [14]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [15]:
chat_history = {}

In [16]:
def get_session_history(session_id : str) -> BaseChatMessageHistory :
    if session_id not in chat_history:
        chat_history[session_id] = InMemoryChatMessageHistory()
    return chat_history[session_id]

In [17]:
config = {"configurable" : {"session_id" : "firstchat"}}

In [19]:
model_with_memory = RunnableWithMessageHistory(llm_google , get_session_history)

In [20]:
model_with_memory.invoke( ("hello , i am akash") , config=config ).content

'Hello Akash! How can I help you today?'

In [21]:
model_with_memory.invoke( ("hello , tell my name") , config=config ).content

'Your name is Akash.'

TOOL

In [29]:
# Wikipedia Tool

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

#  WikipediaAPIWrapper :Utility class allows you to fetch summaries, full page content, and search results from Wikipedia.
#  WikipediaQueryRun : tool wrapper that allows you to execute Wikipedia searches within LangChain.

wiki_api_wrapper = WikipediaAPIWrapper()

wiki_tool = WikipediaQueryRun(api_wrapper = wiki_api_wrapper)

print(wiki_tool.name)
print(wiki_tool.description)
print(wiki_tool.args)

wikipedia
A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.
{'query': {'description': 'query to look up on wikipedia', 'title': 'Query', 'type': 'string'}}


In [30]:
print(wiki_tool.run( { "query" : "Langchain" } ) )

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information.  Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves lar

In [31]:
#Youtube search tool
from langchain_community.tools import YouTubeSearchTool

yt_tool = YouTubeSearchTool()

print(yt_tool.name)
print(yt_tool.description)
print(yt_tool.args)

youtube_search
search for youtube videos associated with a person. the input to this tool should be a comma separated list, the first part contains a person name and the second a number that is the maximum number of video results to return aka num_results. the second part is optional
{'query': {'title': 'Query', 'type': 'string'}}


In [32]:
print(yt_tool.run("Campusx"))

['https://www.youtube.com/watch?v=Cs88FbH9sSU&pp=ygUHQ2FtcHVzeA%3D%3D', 'https://www.youtube.com/shorts/tFvfkiMXgIw']


In [33]:
# For web Search

from langchain_community.tools.tavily_search import TavilySearchResults

websearch_tool = TavilySearchResults()

websearch_tool.invoke( {"query" : "How to master Langgraph" } )



[{'title': 'Mastering LangGraph: The Ultimate Guide to AI Systems, RAG ...',
  'url': 'https://medium.com/@piyushkashyap045/mastering-langgraph-the-ultimate-guide-to-ai-systems-rag-agents-and-tools-ceca33090ef5',
  'content': 'Tools can be customized or built from scratch using LangGraph’s framework.\nHands-On: Setting Up LangGraph\nTo get started with LangGraph, follow these steps:\n1. Install Prerequisites\nEnsure you have Python 3.9+ installed. Create a virtual environment and install the required libraries:\npip install langchain chromadb google-generative-ai\n2. Set Up Your Project\nCreate a .env file to store API keys and other environment variables. Use a tool like dotenv to manage them: [...] from dotenv import load_dotenv\nimport os\nload_dotenv()\nAPI_KEY = os.getenv("YOUR_API_KEY")\n3. Initialize LangGraph\nSet up LangGraph with basic configurations:\nfrom langchain.agents import initialize_agent\nfrom langchain.tools import WikipediaQueryRunner\nfrom langchain.llms import G

Basic Langchain agent (old version classes)

In [34]:
from langchain.agents import AgentType
from langchain.agents import initialize_agent
from langchain.agents import load_tools

In [39]:
tool = load_tools(["wikipedia"] , llm = llm_google)
agent = initialize_agent( tool , llm_google , agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION , verbose = True)


In [40]:
agent.run("tell me india's population")



> Entering new AgentExecutor chain...
Thought: I can use wikipedia to find the population of India.

Action: wikipedia
Action Input: population of India
Observation: Page: Demographics of India
Summary: India is the most populous country in the world with one-sixth of the world's population.
According to estimates from the United Nations (UN), India has overtaken China as the country with the largest population in the world, with a population of 1,425,775,850 at the end of April 2023.
Between 1975 and 2010, the population doubled to 1.2 billion, reaching the billion mark in 2000. According to the UN's World Population dashboard, India's population now stands at slightly over 1.428 billion, edging past China's population of 1.425 billion people, as reported by the news agency Bloomberg. In 2015, India's population was predicted to reach 1.7 billion by 2050. In 2017 its population growth rate was 0.98%, ranking 112th in the world; in contrast, from 1972 to 1983, India's population grew

"India's population is estimated to be over 1.428 billion."